# EXAMEN DATASCIENCE II_Resuelto

Antes de nada realizar un git clone del proyecto de github. Para la realización del test, borrar las opciones incorrectas dejando sólamente la respuesta correcta

# ARIKETA 1 (2 puntos)

Elige la respuesta mas adecuada para las siguientes preguntas. Nota: Respuesta correcta +0.20. Respuesta incorrecta -0.10

### NOTA: Llegado a este punto, tienes que realizar un git commit y un git push. Esta parte del notebook no se puede modificar y no se puede empezar con el ejercicio 2 sin haber hecho los pasos previos. Si se hace un cambio en el test a lo largo del examen, no se corregirán los ejercicios restantes.

# ARIKETA 2 (2 puntos)

Dada la página web https://dreguera.github.io y utilizando la Distancia de Levenshtein, printear en pantalla cuantas veces aparece el término escrito correctamente

**Librerías**

In [2]:
from itertools import chain
import re
import string
import unidecode
import nltk
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize, sent_tokenize

from lxml import html
import requests
import codecs

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import Levenshtein
import gensim

import pandas as pd
import seaborn as sns
import numpy as np

**Cargamos datos**

In [172]:
link_web_principal = 'https://dreguera.github.io/'
codigo_text_página = '//div[@class="wrapper"]/div[@class="box azul_claro"]/text()'

page = requests.get(link_web_principal)
tree = html.fromstring(page.content)
texto_importante = tree.xpath(codigo_text_página) 

codigo_text_página = '//p[@id="muestra"]/text()'

page2 = requests.get(link_web_principal)
tree2 = html.fromstring(page2.content)
elemento = tree2.xpath(codigo_text_página) 


**Se word_tokenizan y se ponen a minúsculas**

In [173]:
words_per_sentence_clean = []
for line in texto_importante:
    words_per_sentence_clean = ([w.lower() for w in word_tokenize(line)])
    
words_per_sentence_clean_comparar = []
for line in elemento:
    words_per_sentence_clean_comparar.append([w.lower() for w in word_tokenize(line)])

**Se mide similitud por distacia levenshtein**

In [174]:
cantidad = []
veces = 0
for i in range(len(words_per_sentence_clean_comparar[0])):
    if Levenshtein.distance(words_per_sentence_clean[0], words_per_sentence_clean_comparar[0][i]) == 0:
        veces += 1
        
print('Basándonos en la distacia Levenshtein, la palabra Scale aparece repetida', veces, 'veces')

Basándonos en la distacia Levenshtein, la palabra Scale aparece repetida 3 veces


# ARIKETA 3 (4 puntos)

Vamos construir un sistema de recomendación de películas basandonos en la similitud del coseno. Para ello cargaremos el csv "movie_dataset.csv" un dataframe

In [175]:
df = pd.read_csv("movie_dataset.csv")
df.head(2)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski


Una vez cargado el dataframe, vamos a realizar un simple preproceso, cambiando en las columnas ['keywords', 'cast', 'genres', 'director'] los valores NaN por ''.


In [176]:
df[['keywords', 'cast', 'genres', 'director']] = df[['keywords', 'cast', 'genres', 'director']].fillna('')

Entre todas estas características, las que nos interesan para encontrar la similitud para hacer la  recomendación son las siguientes: keywords, cast, genres y director. Para ello, vamos a crearnos una nueva columna df['combined_features'] concatenando los strings de las columnas keywords, cast, genres y director.

In [177]:
def combined_features(row):
    return row['keywords']+" "+row['cast']+" "+row['genres']+" "+row['director']
df["combined_features"] = df.apply(combined_features, axis =1)

La función anterior combinará todas nuestras features(palabras clave, elenco, géneros y director) de sus respectivas filas, y devolverá una fila con todas las características combinadas en un solo string.

Una vez creada esta nueva columna df["combined_features"], creamos la matriz de vectores y su matriz de similitud del coseno

Si todo ha ido bien, la matriz de similitud debería tener esta pinta.

In [178]:
vectorizer = CountVectorizer().fit_transform(df["combined_features"].values)
vectors = vectorizer.toarray()
csim = cosine_similarity(vectors)

In [179]:
csim

array([[1.        , 0.10540926, 0.12038585, ..., 0.        , 0.        ,
        0.        ],
       [0.10540926, 1.        , 0.0761387 , ..., 0.03651484, 0.        ,
        0.        ],
       [0.12038585, 0.0761387 , 1.        , ..., 0.        , 0.11145564,
        0.        ],
       ...,
       [0.        , 0.03651484, 0.        , ..., 1.        , 0.        ,
        0.04264014],
       [0.        , 0.        , 0.11145564, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.04264014, 0.        ,
        1.        ]])

Ahora, debeis obtener del dataframe el index del título de la película escrapeado de la página https://dreguera.github.io. Acordaos de utilizar df.values para obtener el valor concreto.

**Scrapeo**

In [180]:
link_web_principal = 'https://dreguera.github.io/'
codigo_text_página = '//div[@id="ariketa2"]/div[@class="box azul_claro"]/text()'

page = requests.get(link_web_principal)
tree = html.fromstring(page.content)
pelicula = tree.xpath(codigo_text_página)
pelicula

['The Silence of the Lambs']

**Índice de la película**

In [181]:
loc = df[df['original_title'] == pelicula[0]]
localizacion = loc.index
print(pelicula[0] + ':',localizacion[0])

The Silence of the Lambs: 2091


Esta función la debeis utilizar para obtener el índice del nombre de esta película. El índice lo deberíais guardar en una variable llamada por ejemplo movie_index

Los indices os deberían dar algo así:

Ahora, vamos a buscar las películas similares en la matriz de similitud. Para ello vamos a utilizar la siguiente función que me devuelve los indices de las películas y el coeficiente de similitud.



Para ello usaremos el movie_index de la película que hemos escrapeado. El método enumerate() agregará un contador a la lista iterable cosine_sim y la devolverá en forma de lista de películas similares con el puntaje de similitud de cada índice.

**Valor de películas similares, sin ordenar**

In [182]:
similar_movies = list(enumerate(csim[localizacion[0]])) 
similar_movies

[(0, 0.0),
 (1, 0.0),
 (2, 0.16681153124565984),
 (3, 0.15689290811054724),
 (4, 0.12000000000000002),
 (5, 0.0),
 (6, 0.0),
 (7, 0.07302967433402215),
 (8, 0.0),
 (9, 0.08164965809277262),
 (10, 0.0),
 (11, 0.08000000000000002),
 (12, 0.0),
 (13, 0.0),
 (14, 0.07559289460184544),
 (15, 0.11766968108291043),
 (16, 0.07184212081070997),
 (17, 0.0),
 (18, 0.0),
 (19, 0.0),
 (20, 0.04082482904638631),
 (21, 0.04082482904638631),
 (22, 0.0),
 (23, 0.0),
 (24, 0.035355339059327376),
 (25, 0.08944271909999159),
 (26, 0.15118578920369088),
 (27, 0.04082482904638631),
 (28, 0.04082482904638631),
 (29, 0.08528028654224418),
 (30, 0.0),
 (31, 0.04000000000000001),
 (32, 0.11338934190276817),
 (33, 0.11338934190276817),
 (34, 0.0),
 (35, 0.0),
 (36, 0.0),
 (37, 0.0),
 (38, 0.11766968108291043),
 (39, 0.0),
 (40, 0.0),
 (41, 0.03922322702763681),
 (42, 0.0),
 (43, 0.04000000000000001),
 (44, 0.043643578047198484),
 (45, 0.11766968108291043),
 (46, 0.07427813527082075),
 (47, 0.0),
 (48, 0.08944271

Ahora, vamos a ordenar la lista creada similar_movies de manera descendente en base a su coeficiente de similitud del coseno. Para ello utilizar la siguiente función que lo único que hace es ordenar descendentemente de manera rápida y fácil

**Se ordena**

In [183]:
sorted_similar_movies = sorted(similar_movies, key=lambda x:x[1], reverse=True)
sorted_similar_movies

[(2091, 1.0000000000000007),
 (497, 0.40249223594996214),
 (2642, 0.32659863237109044),
 (970, 0.31304951684997057),
 (2072, 0.3055050463303894),
 (779, 0.2984810028978546),
 (180, 0.2919201796799047),
 (1013, 0.2919201796799047),
 (1127, 0.2919201796799047),
 (3834, 0.2919201796799047),
 (3257, 0.27456258919345766),
 (2904, 0.2694301256218254),
 (4680, 0.2683281572999748),
 (724, 0.2618614682831909),
 (1781, 0.2599734734478726),
 (717, 0.25584085962673253),
 (1351, 0.25584085962673253),
 (696, 0.25021729686848976),
 (877, 0.25021729686848976),
 (924, 0.25021729686848976),
 (2958, 0.25021729686848976),
 (3014, 0.25021729686848976),
 (4792, 0.25021729686848976),
 (3199, 0.25),
 (354, 0.24494897427831788),
 (2318, 0.24494897427831788),
 (2716, 0.24494897427831785),
 (3270, 0.243708718337977),
 (135, 0.24000000000000005),
 (622, 0.24000000000000005),
 (1564, 0.23664319132398465),
 (365, 0.23533936216582085),
 (4638, 0.23094010767585035),
 (1328, 0.23094010767585033),
 (2169, 0.23094010767

Ahora lo que debéis hacer es mostrar el título de las 15 primeras películas. Para ello, primero debeis obtener el título del df del indice y luego printearlo. 

**La 15 primeras películas**

In [184]:
def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]

contador = 0 
for movie in sorted_similar_movies: 
    print(get_title_from_index(movie[0]))
    contador += 1
    if contador == 15:
        break

The Silence of the Lambs
Red Dragon
Buffalo Soldiers
Hannibal Rising
Freedom Writers
Dark Water
The Bourne Ultimatum
Child 44
High Crimes
Frenzy
American Psycho
Heaven is for Real
Malevolence
Man on Fire
A Walk Among the Tombstones


# ARIKETA 4 (2 puntos)

Dados los ficheros noticia1.txt y noticia2.txt mediante la librería gensim devolver por pantalla (SÓLO) el porcentaje de similitud entre los dos textos. (Nota: Se pude hacer uso de las librerías vistas en clase para limpieza de texto)

**1 Cargar datos**

In [185]:
file_docs = []
with open ('noticia1.txt') as f:
    tokens = sent_tokenize(f.read())
    for line in tokens:
        file_docs.append(line)
        
file_docs2 = []
with open ('noticia2.txt') as f:
    tokens = sent_tokenize(f.read())
    for line in tokens:
        file_docs2.append(line)

**2 Primera sentence**

In [186]:
sentences = file_docs

words_per_sentence_clean = []
for line in sentences:
    query_doc = [w.lower() for w in word_tokenize(line)]
    words_per_sentence_clean.append([w for w in query_doc if w not in string.punctuation])

**3 Modelos gensim**

In [187]:
words_id_dictionary = gensim.corpora.Dictionary(words_per_sentence_clean)
corpus = [words_id_dictionary.doc2bow(word_id) for word_id in words_per_sentence_clean]

tf_idf = gensim.models.TfidfModel(corpus)
sims = gensim.similarities.Similarity('',tf_idf[corpus],
                                        num_features=len(words_id_dictionary))

**4 Segunda sentence**

In [188]:
sentences2 = file_docs2
for line in sentences2:
    query_doc = [w.lower() for w in word_tokenize(line)]
    query_doc = [w for w in query_doc if w not in string.punctuation]
    query_doc_bow = words_id_dictionary.doc2bow(query_doc)

**5 Resultado**

In [189]:
query_doc_tf_idf = tf_idf[query_doc_bow]
sum_of_sims = np.sum(sims[query_doc_tf_idf], dtype = np.float32)

print('Porcentaje de similitud entre noticia1 y noticia 2 es de %s' % float(sum_of_sims/len(sentences) * 100) + '%')

Porcentaje de similitud entre noticia1 y noticia 2 es de 35.53958833217621%
